In [ ]:
# Instalo las dependencias necesarias
from huggingface_hub import notebook_login

notebook_login()

In [24]:
!pip install transformers datasets accelerate peft bitsandbytes trl --quiet

In [25]:
import torch
torch.cuda.is_available()

True

In [26]:
!pip install --upgrade trl

In [27]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from datasets import Dataset
from trl import SFTTrainer
from peft import get_peft_model, LoraConfig, TaskType

In [29]:
# Preparación del Dataset
# Creamos un conjunto de datos de ejemplo en español que contenga preguntas típicas de asistencia al cliente y sus respuestas.
data = [
    {
        "instruccion": "¿Cuál es el horario de atención al cliente?",
        "respuesta": "Nuestro horario de atención es de lunes a viernes de 9am a 6pm."
    },
    {
        "instruccion": "¿Cómo puedo cancelar mi pedido?",
        "respuesta": "Puedes cancelar tu pedido contactando a nuestro equipo de soporte dentro de las 24 horas posteriores a la compra."
    },
    {
        "instruccion": "¿Qué métodos de pago aceptan?",
        "respuesta": "Aceptamos tarjetas de crédito, débito y PayPal."
    },
    {
        "instruccion": "¿Tienen política de devoluciones?",
        "respuesta": "Sí, contamos con una política de devoluciones de 30 días. Revisa nuestra web para más detalles."
    },
    {
        "instruccion": "¿Cómo puedo rastrear mi envío?",
        "respuesta": "Puedes rastrear tu envío con el número de seguimiento que te enviamos por correo electrónico."
    },
    {
        "instruccion": "¿Cómo puedo actualizar mi información personal?",
        "respuesta": "Para actualizar tus datos personales, ingresa a tu cuenta y selecciona la opción 'Perfil' para editar tu información."
    },
    {   "instruccion": "¿Qué hago si no recibo mi pedido?",
        "respuesta": "Si no has recibido tu pedido, por favor contacta a nuestro equipo de soporte para verificar el estado del envío."
    },
   {
        "instruccion": "¿Puedo modificar mi pedido después de realizarlo?",
        "respuesta": "Una vez realizado el pedido, solo se pueden realizar modificaciones contactando a nuestro equipo de soporte, siempre que el pedido no haya sido procesado."
    },
    {
        "instruccion": "¿Cómo puedo obtener una factura de mi compra?",
        "respuesta": "Puedes solicitar una factura de tu compra a través de la sección 'Mis Compras' en tu cuenta o contactando a nuestro soporte."
    },
    {
        "instruccion": "¿Qué debo hacer si mi producto llega dañado?",
        "respuesta": "Si recibes un producto dañado, contacta a nuestro servicio de atención al cliente inmediatamente para gestionar la devolución o el reemplazo."
    },
    {
        "instruccion": "¿Tienen ofertas o descuentos especiales?",
        "respuesta": "Sí, periódicamente ofrecemos promociones y descuentos especiales. Te recomendamos suscribirte a nuestro boletín para estar informado."
    },   
]


# Unificamos la instrucción y la respuesta en un formato de prompt
for registro in data:
    registro["text"] = (
        f"### Instrucción:\n{registro['instruccion']}\n\n"
        f"### Respuesta:\n{registro['respuesta']}"
    )

# Convertimos la lista en un Dataset de Hugging Face
dataset = Dataset.from_list(data)


In [30]:
# Carga y Configuración del Modelo con QLoRA
# Usamos un modelo preentrenado en español; en este ejemplo empleamos "datificate/gpt2-small-spanish".
# Configuramos la cuantización a 4 bits mediante BitsAndBytes para hacer el entrenamiento más eficiente.
model_name = "datificate/gpt2-small-spanish"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left", truncation=True)
# Si no tiene pad_token, lo añadimos manualmente y redimensionamos las embeddings del modelo
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    
# Aseguramos que el pad_token esté definido
print("Pad Token:", tokenizer.pad_token)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Cargamos el modelo en modo 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,   # Permite ejecutar código remoto si el modelo lo requiere
    device_map="auto",
)

# Si hemos añadido un nuevo pad_token, redimensionamos las embeddings
if tokenizer.pad_token_id is None or tokenizer.pad_token_id >= model.config.vocab_size:
    model.resize_token_embeddings(len(tokenizer))
    
# Adjuntar adaptadores (LoRA) para que el modelo cuantizado sea entrenable
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # Ajustamos para modelos de lenguaje causal
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)
model = get_peft_model(model, peft_config)

Pad Token: <|endoftext|>


In [31]:
# Configuración de los Parámetros de Entrenamiento
# Se definen los argumentos de entrenamiento para ajustar el modelo. 
# Se han seleccionado hiperparámetros razonables, pero puedes ajustar según tus recursos y resultados esperados.
training_args = TrainingArguments(
    output_dir="./resultados",
    num_train_epochs=10,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=20,
    learning_rate=2e-4,
    weight_decay=0.01,
    fp16=True,          # Usamos FP16 para acelerar el entrenamiento en GPUs compatibles
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none",
    save_total_limit=3,
    push_to_hub=False,
)


In [32]:
# Fine-Tuning del Modelo usando SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=100,
    tokenizer=tokenizer,
    args=training_args,
    packing=False,
)

# Inicia el proceso de entrenamiento
trainer.train()

c:\Users\pauci\miniconda3\envs\pytorch_env\lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\pauci\miniconda3\envs\pytorch_env\lib\site-packages\trl\trainer\sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
c:\Users\pauci\miniconda3\envs\pytorch_env\lib\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/11 [00:00<?, ? examples/s]

c:\Users\pauci\miniconda3\envs\pytorch_env\lib\site-packages\trl\trainer\sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
c:\Users\pauci\miniconda3\envs\pytorch_env\lib\site-packages\trl\trainer\sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


  0%|          | 0/10 [00:00<?, ?it/s]

{'train_runtime': 1.5458, 'train_samples_per_second': 71.159, 'train_steps_per_second': 6.469, 'train_loss': 7.514565277099609, 'epoch': 10.0}


TrainOutput(global_step=10, training_loss=7.514565277099609, metrics={'train_runtime': 1.5458, 'train_samples_per_second': 71.159, 'train_steps_per_second': 6.469, 'total_flos': 2857548533760.0, 'train_loss': 7.514565277099609, 'epoch': 10.0})

In [33]:
# Función de Inferencia: Generación de Respuestas
def generar_respuesta(instruccion):
    # Se arma el prompt en el mismo formato del entrenamiento
    prompt = f"### Instrucción:\n{instruccion}\n\n### Respuesta:"
    # Usamos el tokenizer para obtener input_ids y attention_mask
    encoded_input = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    input_ids = encoded_input["input_ids"].to(model.device)
    attention_mask = encoded_input["attention_mask"].to(model.device)
    
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=100,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            no_repeat_ngram_size=2,
            pad_token_id=tokenizer.pad_token_id,
            attention_mask=attention_mask, 
            )
    
    texto_generado = tokenizer.decode(output[0], skip_special_tokens=True)
    if "### Respuesta:" in texto_generado:
        respuesta = texto_generado.split("### Respuesta:")[-1].strip()
    else:
        respuesta = texto_generado.strip()
    return respuesta

In [34]:
# Ejemplos de Uso del Chatbot
pregunta_cliente = "¿Cómo puedo cambiar mi dirección de envío?"
respuesta = generar_respuesta(pregunta_cliente)

print("Pregunta del Cliente:")
print(pregunta_cliente)
print("\nRespuesta del Chatbot:")
print(respuesta)



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Pregunta del Cliente:
¿Cómo puedo cambiar mi dirección de envío?

Respuesta del Chatbot:
¡Dáviento!
!# ##
El poder de la mente es una idea que un individuo está siendo capaz de usar un lenguaje humano.
Es una forma de pensamiento con sentido y una filosofía de mente. Es un concepto que se caracteriza por su carácter de pensar y no tener sentido. El sujeto es un sujeto, un tipo, una mente, y la forma


In [35]:
otras_preguntas = [
    "¿Cuáles son los métodos de pago disponibles?",
    "¿Tienen política de devoluciones extendida?",
    "¿A qué hora cierran sus oficinas de atención?"
]

for pregunta in otras_preguntas:
    print("\nPregunta:")
    print(pregunta)
    print("Respuesta:")
    print(generar_respuesta(pregunta))


Pregunta:
¿Cuáles son los métodos de pago disponibles?
Respuesta:
Los métodos usados para las actividades en línea, en la mayoría de los países, son a menudo aquellos que se han interesado en las habilidades de cualquier persona o entidad. Por ejemplo, los medios de comunicación de la época, tales como radio, televisión, periódicos, revistas, etc., los autores de obras de teatro, las historias, películas, cine, videojuegos, literatura

Pregunta:
¿Tienen política de devoluciones extendida?
Respuesta:
"¿Asesoramiento de una nueva dimensión"
- “¿Quién ha vuelto a la izquierda? 
"En el debate de los candidatos de la coalición "Núcleo" y "Los liberales" de 2004, el grupo de "Nacionalistas" declaró: "No se trata de un grupo, sino de personas que quieren convertirse en un partido político.

Pregunta:
¿A qué hora cierran sus oficinas de atención?
Respuesta:
¿Qué qué se siente??;
 ## Revisar: No.
Maude es una serie de televisión estadounidense de comedia, escrita por el director estadounidense

In [36]:
# 8. Guardado del Modelo y Tokenizador
model.save_pretrained("chatbot_asistencia_cliente")
tokenizer.save_pretrained("chatbot_asistencia_cliente")

('chatbot_asistencia_cliente\\tokenizer_config.json',
 'chatbot_asistencia_cliente\\special_tokens_map.json',
 'chatbot_asistencia_cliente\\vocab.json',
 'chatbot_asistencia_cliente\\merges.txt',
 'chatbot_asistencia_cliente\\added_tokens.json',
 'chatbot_asistencia_cliente\\tokenizer.json')